This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [31]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.20.0


In [32]:
# Imports consolidated in the first code cell above.
# This cell is intentionally left minimal to avoid duplicate imports.


In [33]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "train_1"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "test_1"    # e.g. './muffin-vs-chihuahua/test'

In [34]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [35]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [36]:
# SIMPLE CNN MODEL ARCHITECTURE (improved with regularization, batchnorm and dropout)

initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

from tensorflow.keras import regularizers

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4), input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


In [37]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [38]:
# TRAINING THE CNN (with callbacks: early stopping and model checkpoint to save best model)
MODEL_PATH = 'exercise_6_trained_model_improved.h5'
EPOCHS = 15  # Reduced training epochs as requested

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(MODEL_PATH, save_best_only=True)
]

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks
)


Epoch 1/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6617 - loss: 3.8044

119/119 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - accuracy: 0.7199 - loss: 1.7278 - val_accuracy: 0.5407 - val_loss: 10.8713
Epoch 2/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7938 - loss: 0.5601

119/119 ━━━━━━━━━━━━━━━━━━━━ 232s 2s/step - accuracy: 0.8123 - loss: 0.5352 - val_accuracy: 0.5407 - val_loss: 3.5522
Epoch 3/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8366 - loss: 0.4829

119/119 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.8347 - loss: 0.4879 - val_accuracy: 0.5407 - val_loss: 2.9447
Epoch 4/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 981ms/step - accuracy: 0.8409 - loss: 0.4766

119/119 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - accuracy: 0.8445 - loss: 0.4705 - val_accuracy: 0.6466 - val_loss: 1.1034
Epoch 5/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8561 - loss: 0.4279

119/119 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - accuracy: 0.8627 - loss: 0.4306 - val_accuracy: 0.8032 - val_loss: 0.5906
Epoch 6/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - accuracy: 0.8603 - loss: 0.4211 - val_accuracy: 0.7185 - val_loss: 3.2211
Epoch 7/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - accuracy: 0.8601 - loss: 0.4643 - val_accuracy: 0.6444 - val_loss: 1.1357
Epoch 8/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 778ms/step - accuracy: 0.8644 - loss: 0.4487

119/119 ━━━━━━━━━━━━━━━━━━━━ 105s 884ms/step - accuracy: 0.8765 - loss: 0.4263 - val_accuracy: 0.8974 - val_loss: 0.3730
Epoch 9/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 2304s 20s/step - accuracy: 0.8933 - loss: 0.3961 - val_accuracy: 0.6265 - val_loss: 1.4281
Epoch 10/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 787ms/step - accuracy: 0.8881 - loss: 0.3886

119/119 ━━━━━━━━━━━━━━━━━━━━ 106s 895ms/step - accuracy: 0.8867 - loss: 0.4044 - val_accuracy: 0.9143 - val_loss: 0.3720
Epoch 11/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 104s 872ms/step - accuracy: 0.8999 - loss: 0.3819 - val_accuracy: 0.6667 - val_loss: 0.8409
Epoch 12/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 115s 966ms/step - accuracy: 0.9013 - loss: 0.3752 - val_accuracy: 0.7302 - val_loss: 0.5344
Epoch 13/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 978ms/step - accuracy: 0.9011 - loss: 0.3736

119/119 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.9021 - loss: 0.3658 - val_accuracy: 0.9270 - val_loss: 0.2782
Epoch 14/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - accuracy: 0.9081 - loss: 0.3503 - val_accuracy: 0.7111 - val_loss: 0.7604
Epoch 15/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.8976 - loss: 0.3652 - val_accuracy: 0.8984 - val_loss: 0.3492


In [39]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 18s 492ms/step - accuracy: 0.8801 - loss: 0.3689
Test Accuracy: 0.8800675868988037


In [40]:
# SAVE THE MODEL (also saved by ModelCheckpoint). This ensures a copy with the requested name exists.
model.save('exercise_6_trained_model_improved.h5')
print('Saved model as exercise_6_trained_model_improved.h5')

Saved model as exercise_6_trained_model_improved.h5


In [41]:
# SIMPLE INFERENCE SCRIPT (returns label and confidence)
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    return label, float(pred)

def pretty_predict(img_path, model_path='exercise_6_trained_model_improved.h5'):
    label, conf = predict_image(img_path, model_path=model_path)
    print(f"{img_path} -> {label} (confidence: {conf:.2f})")


In [42]:
# Example usage for the two run_1 images (prints label and confidence)
pretty_predict("run_1/run_1.jpg")
pretty_predict("run_1/run_2.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
run_1/run_1.jpg -> Muffin (confidence: 0.24)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
run_1/run_2.jpg -> Chihuahua (confidence: 1.00)


In [43]:
# Run predictions programmatically for run_1 images and show confidence scores
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model('exercise_6_trained_model_improved.h5')
img_paths = ['run_1/run_1.jpg', 'run_1/run_2.jpg']
for p in img_paths:
    if not os.path.exists(p):
        print('Image not found:', p); continue
    img = image.load_img(p, target_size=IMG_SIZE)
    arr = image.img_to_array(img) / 255.0
    arr = np.expand_dims(arr, 0)
    pred = model.predict(arr)[0,0]
    label = 'Chihuahua' if pred >= 0.5 else 'Muffin'
    print(p, '->', label, f'(confidence: {pred:.2f})')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
run_1/run_1.jpg -> Muffin (confidence: 0.24)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
run_1/run_2.jpg -> Chihuahua (confidence: 1.00)


In [44]:
# Optional: git helper to add and push the trained model to your repository (run only if this folder is a git repo and you have remote configured)
import subprocess, os
model_path = 'exercise_6_trained_model_improved.h5'
if os.path.exists(model_path):
    print('Model exists:', model_path)
    try:
        subprocess.check_call(['git','add',model_path])
        subprocess.check_call(['git','commit','-m','Add improved trained model exercise_6_trained_model_improved.h5'])
        subprocess.check_call(['git','push'])
        print('Attempted to push model to remote. Check your GitHub repo or commit history.')
    except subprocess.CalledProcessError as e:
        print('Git command failed (are you in a repo with a configured remote?):', e)
else:
    print('Model file not found. Train the model or adjust path.')

Model exists: exercise_6_trained_model_improved.h5
Git command failed (are you in a repo with a configured remote?): Command '['git', 'add', 'exercise_6_trained_model_improved.h5']' returned non-zero exit status 128.
